In [ ]:
import nltk
from flask import jsonify, request,Flask
import os
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from tensorflow.keras.optimizers import SGD
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import numpy as np
import pandas as pd
from keras.models import load_model

import json
import random



def clean_up_sentence(sentence):
    print('clean up sentense start')
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print('clean up sentense end')
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence



def bow(sentence, words, show_details=True):
    # tokenize the pattern
    print('bow start')
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    print('bow end')
    return(np.array(bag))



def predict_class(sentence, model):
    # filter out predictions below a threshold
    print('predict class start')
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print('predict class end')
    return return_list



def getResponse(ints):
    print('get response start')
    tag = ints[0]['intent']
    for i in d.keys():
        if(i == tag):
            result = random.choice(d[i])
            break
    print('get response end')
    return result




appFlask = Flask(__name__)



model = load_model('chatbot_model1.h5')
words = pickle.load(open('words1.pkl','rb'))
classes = pickle.load(open('classes1.pkl','rb'))
d = pickle.load(open('Intents.pkl','rb'))




@appFlask.route('/chatbot',methods=['POST'])



def chatbot():
    try:
        input_data=request.get_json(force=True)
        #return 'we have the good score'+str(score)
        input_data = input_data["Input"]
        print("Input:  ",input_data)
        print('chatbot_response start')
        ints = predict_class(input_data, model)
        res = getResponse(ints)
        res = str(res)
        return(res)
        print('chatbot_response end')
        return "OK"
    except:
        return "Answer not found"
    
if __name__ == "__main__":
    appFlask.run(host=os.getenv('IP', '0.0.0.0'),
            port=int(os.getenv('PORT', 5413))) # 1830

has context menu